# Classification Process

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Importing the Necessary Libraries<br></p>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

import time

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Creating a Pandas DataFrame from a CSV file<br></p>

In [2]:
def CreateDataFrame(types,subtype, level):
    data = pd.read_csv('../Extraction/'+types+"/"+subtype+"/"+level+'.csv')
    data["Class"] = level
    return data

In [3]:
def LoadFullData(types,subtype):
    if(subtype=="All"):
        frames = [CreateDataFrame(types,"Roof","Clean"), 
              CreateDataFrame(types,"Roof","Dirty (0-20)"), 
              CreateDataFrame(types,"Roof","Dirty (20-40)"), 
              CreateDataFrame(types,"Roof","Dirty (40-60)"),
              CreateDataFrame(types,"Roof","Dirty (60-80)"),
              CreateDataFrame(types,"Roof","Dirty (80-100)"),
              CreateDataFrame(types,"Grass","Clean"), 
              CreateDataFrame(types,"Grass","Dirty (0-20)"), 
              CreateDataFrame(types,"Grass","Dirty (20-40)"), 
              CreateDataFrame(types,"Grass","Dirty (40-60)"),
              CreateDataFrame(types,"Grass","Dirty (60-80)"),
              CreateDataFrame(types,"Grass","Dirty (80-100)")]
    else:
        frames = [CreateDataFrame(types,subtype,"Clean"), 
              CreateDataFrame(types,subtype,"Dirty (0-20)"), 
              CreateDataFrame(types,subtype,"Dirty (20-40)"), 
              CreateDataFrame(types,subtype,"Dirty (40-60)"),
              CreateDataFrame(types,subtype,"Dirty (60-80)"),
              CreateDataFrame(types,subtype,"Dirty (80-100)")]
    df = pd.concat(frames)
    #Randomize
    df = shuffle(df)
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    return X,y

In [4]:
def LoadMinimizedData(types,subtype, count):
    df = pd.read_csv('../Selection/Selection-'+types+"-"+subtype+"-"+count+".csv")
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    return X,y

In [5]:
def loader(types,subtype,datasize):
    if(datasize=="Full"):
        X,y = LoadFullData(types,subtype)
    else:
        count = datasize.split('-')[-1]
        X,y = LoadMinimizedData(types,subtype,count)
    return X,y

# Classifications

## Optimize KNN Classfiers

In [6]:
def optimizeKNN(X_train, y_train,cv):
    param_grid = [{'clf__weights': ["distance"], 'clf__n_neighbors': [5]}]
    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', KNeighborsClassifier())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    end = time.time()
    return grid_search

## Optimize SVM Classfiers

In [7]:
def optimizeSVM(X_train, y_train,cv):
    param_grid = [
            {'clf__kernel': ['poly'],'clf__degree':[1,5],'clf__coef0':[5,10]}
            #,{'clf__kernel': ['rbf'],'clf__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]}
            #,{'clf__kernel': ['sigmoid'],'clf__coef0':[1,5,10,15,20,30]}
            #,{'clf__kernel':['linear'],'clf__C':[1,5,10]}
            ]

    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', SVC())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Optimize Random Forest Classfiers

In [8]:
def optimizeRandomForest(X_train, y_train,cv):
    param_grid = [
            {'clf__n_estimators': [30],'clf__max_features':['auto']},
            ]

    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', RandomForestClassifier())])
        
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Optimize NN Classifiers 

In [9]:
def optimizeNN(X_train, y_train,cv):
    param_grid = [{'clf__hidden_layer_sizes': [(100,50,30,)]}]
    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', MLPClassifier())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Cycle Work

In [10]:
def cycleWork(X_train, X_test, y_train, y_test,optimize,cv):
    gridsearch = optimize(X_train, y_train,cv)
    traintime = gridsearch.cv_results_['mean_fit_time'].mean()*1000
    print("Mean Fit Time", traintime)
    print("Mean Score Time", gridsearch.cv_results_['mean_score_time'].mean()*1000)

    #print(gridsearch.best_params_)
    estimator = gridsearch.best_estimator_
    params = estimator.get_params()['clf']
    print(params)
    y_train_pred = estimator.predict(X_train)
    trnscore = accuracy_score(y_train, y_train_pred)
    start = time.time()
    y_tst_pred = estimator.predict(X_test)
    end = time.time()
    diff = end-start
    clfperms = X_train.shape[0]/((diff*1000)+0.000000001)
    tstscore = accuracy_score(y_test, y_tst_pred)
    cm = confusion_matrix(y_test, y_tst_pred)
    true_pos = np.diag(cm) 
    precision = np.sum(true_pos / np.sum(cm, axis=0))
    recall = np.sum(true_pos / np.sum(cm, axis=1))
    f1 = 2 * ((precision* recall)/(precision + recall))
    #f1None = f1_score(y_test, y_tst_pred, average=None)
    #f1micro = f1_score(y_test, y_tst_pred, average='micro')
    return [trnscore, tstscore, f1,traintime,clfperms,cm,params]

In [11]:
def CrossBackgroundWork(types,trainsubtype,testsubtype,sizeDict, cycles = 5, out=False,cross_validation=5):    
    data = []
    for datasize in sizeDict:
        print(datasize)
        classifiersDict = {
                        "KNN":optimizeKNN,
                        "SVM":optimizeSVM,
                        "RF":optimizeRandomForest,
                        "NN":optimizeNN
                          }
        for clf in classifiersDict:
            for i in range(0,cycles):
                X_train,y_train = loader(types,trainsubtype,datasize)
                X_test,y_test = loader(types,testsubtype,datasize)
                print("Size ",datasize, " Algorithm ",clf, " Trial ",i)
                acc = cycleWork(X_train, X_test, y_train, y_test,classifiersDict[clf],cross_validation)
                print("Train Accuracy ",acc[0],", Test Accuracy",acc[1], "F1 ",acc[2])
                data.append([clf,datasize.split('-')[-1],i,acc[0],acc[1],acc[2],acc[3],acc[4],acc[5],acc[6]])
                print("########################################")
    numpyarray = np.array(data)
    df = pd.DataFrame(numpyarray, columns = ["Classifier","Count","Cycle","Train Acc", "Test Acc", "F1 Score","Train Time (ms)","clfs per ms","Confusion Matrix","Params"])
    if(out):
        df.to_csv("All-"+types+'-train-'+trainsubtype+"-test-"+testsubtype+'.csv',index = False)
    df.head(10)

In [12]:
types = "Cropped"
trainsubtype = "Grass"
testsubtype = "Roof"
cycles = 2
sizeDict = {
                "Full":None,
                "Min-10":10,
                }
CrossBackgroundWork(types,trainsubtype,testsubtype,sizeDict,cycles = cycles, out=True,cross_validation=5)
trainsubtype = "Roof"
testsubtype = "Grass"
CrossBackgroundWork(types,trainsubtype,testsubtype,sizeDict,cycles = cycles, out=True,cross_validation=5)

Full
Size  Full  Algorithm  KNN

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    3.0s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.1s finished


  Trial  0
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Mean Fit Time 19.87462043762207
Mean Score Time 33.73298645019531
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
Train Accuracy  1.0 , Test Accuracy 0.42603773584905663 F1  3.0411096692332045
########################################
Size  Full  Algorithm  KNN  Trial  1
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    4.1s remaining:    6.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.9s finished


Mean Fit Time 19.7629451751709
Mean Score Time 46.11411094665527
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
Train Accuracy  1.0 , Test Accuracy 0.42603773584905663 F1  3.0411096692332045
########################################
Size  Full  Algorithm  SVM  Trial  0
Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    8.2s remaining:   12.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   14.0s finished


Mean Fit Time 4354.207277297974
Mean Score Time 6.188249588012695
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Train Accuracy  0.9535849056603773 , Test Accuracy 0.24283559577677225 F1  nan
########################################
Size  Min-10  Algorithm  NN  Trial  1
Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\eng_w\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   12.9s remaining:   19.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   18.5s finished


Mean Fit Time 5406.453990936279
Mean Score Time 5.886697769165039
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Train Accuracy  0.9475471698113207 , Test Accuracy 0.27111613876319757 F1  nan
########################################


C:\Users\eng_w\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
